In [21]:
from pathlib import Path
import os
import json
import os
import subprocess
import shutil
from tqdm import tqdm
import csv
import ast
# 读取配置文件
with open('/home/users/hcdai/AI-peptide/RunRosetta/config_chai.json', 'r') as f:
    config = json.load(f)

# 打印配置文件内容
print(config)

path = config['path']
parameter = config['parameter']
# # 设定工作区路径
# os.chdir(path["workspace"])

# getcwd = os.getcwd()
# print("Current working directory: ", getcwd)

{'path': {'workspace': '/home/users/hcdai/AI-peptide/ChaiRosettaScore', 'chai-1': {'ChaiRunPython': '/home/users/hcdai/AI-peptide/ChaiRosettaScore/ChaiRunPython.sh', 'home': '/home/users/hcdai/AI-peptide/Chai-1/chai-lab', 'chaiworkdir': '/home/users/hcdai/AI-peptide/Chai-1/chai-lab/RMSD_Time-test', 'chai_python_executable': '/home/users/hcdai/miniconda3/envs/Chai-1/bin/python'}, 'rosetta': {'score_executable': '/home/users/hcdai/AI-peptide/rosetta_interface_analysis/rosetta.binary.ubuntu.release-371/main/source/bin/score_jd2.static.linuxgccrelease', 'InterfaceAnalyzer': '/home/users/hcdai/AI-peptide/rosetta_interface_analysis/rosetta.binary.ubuntu.release-371/main/source/bin/InterfaceAnalyzer.static.linuxgccrelease', 'pack_input_options': '/home/users/hcdai/AI-peptide/ChaiRosettaScore/setting_options.ini'}, 'ligand_database': '/home/users/hcdai/AI-peptide/ChaiRosettaScore/ligand', 'receptor_database': '/home/users/hcdai/AI-peptide/ChaiRosettaScore/receptor', 'temp': '/home/users/hcdai/

In [22]:
def ligand_processor(csv_path):
    # 定义一个空字典来存储结果
    ligand_dict = {}
    
    # 打开CSV文件
    with open(csv_path, mode='r', newline='', encoding='utf-8') as file:
        # 创建CSV读取器
        csv_reader = csv.DictReader(file)
        
        # 遍历CSV文件的每一行
        for row in csv_reader:
            # 获取ligand_name的值作为键
            ligand_name = row['ligand_name']
            
            # 获取ligand_sequence的值，并尝试将其从字符串解析为字典
            try:
                ligand_sequence = ast.literal_eval(row['ligand_sequence'])
            except (ValueError, SyntaxError) as e:
                print(f"Error parsing ligand_sequence for {ligand_name}: {e}")
                continue  # 或者可以根据需要处理错误
            
            # 将解析后的ligand_sequence存入字典中
            ligand_dict[ligand_name] = ligand_sequence
    
    return ligand_dict
# # 以下代码为测试该函数用
# file_path = '/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_1.0/dataset/standard_csv_withpath/TF_ori_withpath.csv'  # 替换为你的CSV文件路径
# result_dict = ligand_processor(file_path)
 
# # 打印结果
# if result_dict is not None:
#     print(result_dict)
# else:
#     print("No ligand found in the CSV file.")

In [23]:
def receptor_processor(csv_path):
    # 定义一个空字典来存储结果
    receptor_dict = {}
    
    # 打开CSV文件
    with open(csv_path, mode='r', newline='', encoding='utf-8') as file:
        # 创建CSV读取器
        csv_reader = csv.DictReader(file)
        
        # 遍历CSV文件的每一行
        for row in csv_reader:
            # 获取receptor_name的值作为键
            receptor_name = row['receptor_name']
            
            # 获取receptor的值，并尝试将其从字符串解析为字典
            try:
                receptor_sequence = ast.literal_eval(row['receptor_sequence'])
            except (ValueError, SyntaxError) as e:
                print(f"Error parsing receptor_sequence for {receptor_name}: {e}")
                continue  # 或者可以根据需要处理错误
            
            # 将解析后的ligand_sequence存入字典中
            receptor_dict[receptor_name] = receptor_sequence
    
    return receptor_dict
# # 以下代码为测试该函数用
# file_path = '/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_1.0/dataset/standard_csv_withpath/TF_ori_withpath.csv'  # 替换为你的CSV文件路径
# result_dict = receptor_processor(file_path)
 
# # 打印结果
# if result_dict is not None:
#     print(result_dict)
# else:
#     print("No receptor found in the CSV file.")


In [24]:
def write_fasta_for_protein(protein_name, ligand_chains, receptor_chains, output_dir):
    """
    为单个蛋白质写入FASTA文件，包含配体和受体链信息。
 
    参数:
    protein_name (str): 蛋白质的名称。
    ligand_chains (dict): 配体链名称到序列的映射。
    receptor_chains (dict): 受体链名称到序列的映射。
    output_dir (str): 输出FASTA文件的目录。
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    fasta_file_path = os.path.join(output_dir, f"{protein_name}.fasta")
    
    with open(fasta_file_path, 'w') as f:
        # 写入配体信息
        for chain_name, sequence in ligand_chains.items():
            f.write(f'>protein|name={chain_name}\n{sequence}\n')
        
        # 写入受体信息
        # 可选：为受体链名称添加前缀或后缀以区分它们
        for chain_name, sequence in receptor_chains.items():
            receptor_chain_name = f'_R_{chain_name}'  # 添加前缀 '_R' 以区分受体链
            f.write(f'>protein|name={receptor_chain_name}\n{sequence}\n')
 
def write_fasta_files_from_dicts(ligand_dict, receptor_dict, output_dir):
    """
    从配体和受体词典中为每个蛋白质写入FASTA文件。
 
    参数:
    ligand_dict (dict): 配体信息的词典，键为蛋白质名称，值为链名称到序列的映射。
    receptor_dict (dict): 受体信息的词典，键为蛋白质名称，值为链名称到序列的映射。
    output_dir (str): 输出FASTA文件的目录。
    """
    for protein_name in ligand_dict:
        if protein_name in receptor_dict:
            write_fasta_for_protein(protein_name, ligand_dict[protein_name], receptor_dict[protein_name], output_dir)
        else:
            print(f"Warning: No receptor information found for protein {protein_name}")
 
# 示例词典
ligand_example = {
    'protein1': {'chain1': 'ATGCGTACGT', 'chain2': 'CGTACGTAGC'},
    'protein2': {'chainA': 'GCTAGCTAGC', 'chainB': 'AGCTAGCTAG'}
}
 
receptor_example = {
    'protein1': {'chainX': 'TAGCTAGCTA', 'chainY': 'GCTAGCTAGC'},
    'protein2': {'chain1': 'CGTAGCTAGC', 'chain2': 'AGCTAGCTCG'}  # 注意这里chain2与ligand_example中的chain2不是同一个链，但名称相同
}
 
# 输出目录
output_directory = '/home/users/hcdai/AI-peptide/RunRosetta/test_output_fasta'
 
# 写入FASTA文件
write_fasta_files_from_dicts(ligand_example, receptor_example, output_directory)

In [15]:
# 将数据库的序列信息写入fasta文件
csv_path="/home/users/hcdai/AI-peptide/RunRosetta/TF_ori_withpath.csv"
ligand_dict=ligand_processor(csv_path)
receptor_dict=receptor_processor(csv_path)
output_directory = '/home/users/hcdai/AI-peptide/RunRosetta/test_output_fasta'
write_fasta_files_from_dicts(ligand_dict, receptor_dict, output_directory)

In [ ]:
def target_sequence_generator(temp_dir=path["temp"],chaiworkdir=path["chaiworkdir"]):
    
    output_dir=Path(os.path.join(temp_dir, "cif"))
    # 目标序列文件
    target_fasta_file = Path("/home/users/hcdai/AI-peptide/RunRosetta/test_output_fasta")
    # 目标脚本
    script = f'''from pathlib import Path
import numpy as np
import torch
import os
from chai_lab.chai1 import run_inference
output_dir=Path("{output_dir}")
candidates = run_inference(
    fasta_file=Path("{target_fasta_file}"),
    output_dir=output_dir,
    # 'default' setup
    num_trunk_recycles=3,
    num_diffn_timesteps=200,
    seed=42,
    device=torch.device("cuda:0"),
    use_esm_embeddings=True,
)
cif_paths = candidates.cif_paths
scores = [rd.aggregate_score for rd in candidates.ranking_data]
# Load pTM, ipTM, pLDDTs and clash scores for sample 2
scores = np.load(output_dir.joinpath("scores.model_idx_2.npz"))'''  
    
    # 将目标脚本写入chaiworkdir下的run.py文件
    run_file = Path(os.path.join(chaiworkdir, "run.py"))
    with open(run_file, 'w') as f:
        f.write(script)

    # 成功写入脚本文件
    print(f"Chai script generated at {run_file}.")

    return run_file, output_dir, target_fasta_file

SyntaxError: invalid syntax (3258237063.py, line 3)

In [ ]:
# 定义运行chai脚本函数
def run_chai_script(chai_python_path,run_file):
    # # 使用subprocess运行chai脚本
    # run_file = str(run_file)
    # command = [chai_python_path, run_file]
    # print("command:", command)    
    # process = subprocess.run(command,shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # # 打印输出和错误信息
    # print("输出:", process.stdout.decode())
    # print("错误:", process.stderr.decode())

    # 使用os.system运行chai脚本
    run_file = str(run_file)
    command = f"{chai_python_path} {run_file}"
    os.system(command)

    return True

# 定义运行bash脚本函数
def run_bash_script(bash_script_path):
    # 使用subprocess运行bash脚本
    bash_script_path = str(bash_script_path)

    return True

In [ ]:
def convert_cif_to_pdb(cif_file_path,pdb_file_path):
    # 读取cif文件转换为pdb文件
    if not os.path.exists(pdb_file_path):
        os.makedirs(pdb_file_path)
    # 定义输入和输出目录
    input_cif_directory = cif_file_path  
    output_pdb_directory = pdb_file_path  

    # 获取输入目录中所有CIF文件的列表并进行排序
    cif_files = [f for f in os.listdir(input_cif_directory) if f.endswith(".cif")]
    cif_files.sort()  # 按字母顺序排序

    # 使用tqdm为循环添加进度条
    for cif_file in tqdm(cif_files, desc="Converting files"):
        cif_path = os.path.join(input_cif_directory, cif_file)
        pdb_file = cif_file.replace(".cif", ".pdb")
        pdb_path = os.path.join(output_pdb_directory, pdb_file)

        obabel_path = path["obabel"]
        # 使用Open Babel将CIF转换为PDB
        obabel_command = [obabel_path, cif_path, "-O", pdb_path]
        with open(os.devnull, 'w') as devnull:
            subprocess.run(obabel_command, stdout=devnull, stderr=devnull)

        # 使用awk删除文件头
        awk_command = ["awk", '/^ATOM |^TER/']
        with open(pdb_path, "rb") as input_file:
            awk_process = subprocess.run(awk_command, stdin=input_file, stdout=subprocess.PIPE, text=True)
            tmp2_content = awk_process.stdout

        with open(pdb_path, "w") as tmp2_file:
            tmp2_file.write(tmp2_content)

    print("转换完成")